# Crypto / zerodaycrypto

## Foreword

Wew, there's actually so much I want to say here that I don't think I can say everything. I discovered this result around January 2024, and have been writing the paper on-and-off for more than half a year now, but never really found the inspiration to just properly sit down and finish it. The quick abstract of it is that you can solve the adversarial MIHNP with $1/n$ of the top bits of only $O(n)$ outputs, compared to $O(n^3)$ in the current state-of-the-art.

One of the original plans was to write a paper in time for SEETF, and then release the challenge along with the paper (so it's an implementation challenge). But then SEETF got scrapped. Some time later, I got dragged into SekaiCTF (I don't even remember how) and they said they were accepting zero-days, so here's the end result.

I did not want to set the bounds "too tight" so to speak, especially for a 48-hour CTF. After some deliberation, I went with parameters that allowed me to solve the challenge with only 10 outputs and in under a minute. But I also gave 14 outputs, and infinite time (technically 48 hours) to allow solutions that were "better than the state-of-the-art" without having to be as tight as mine.

In particular, my choice to give the highest $\frac{1}{3}$ of bits was particularly significant because the current state-of-the-art was [this paper](https://ieeexplore.ieee.org/document/10089839) which explicitly stated that for the $\frac{1}{3}$ case you needed 32 outputs (which we don't have) and a lattice of dimension 46441 (which is not practical), implying that their construction would not be sufficient. The intended solve path then would be to try to "filter out" bad polynomials from their lattice to improve its strength.

Nevertheless, multiple players managed to use the paper anyway to solve it directly, proving that crypto players are absolutely insane.
- First blood was `bachpc` from `thehackerscrew`, who used 13 outputs and a 750x750 lattice (45 minutes?)
- Second blood was `ks`, from `ks loves leoneed&mmj`, who used 11 outputs and a 914x914 lattice (1-2 hours maybe?)

Congratulations to them, and anyone else who solved the challenge! While they didn't rediscover my construction, I found their approach insightful, both in terms of the strength of that paper's lattice construction, as well as in understanding crypto players' perserverance and determination in general.

## The intended solution

The brief explanation is that the lattices constructed in that paper contain lots of good polynomials, but also lots of bad ones. If we filter out all the bad ones, you end up with the SBG space. Be sure to have read the [hint on the SBG space](zerodaycrypto_hint.pdf) as a pre-requisite.

What follows in the rest of the notebook is a demonstration of how to construct the good polynomials. The exposition is meant to be more practical than theoretical, so I'll omit or handwave away the proofs and defer that to the paper which I hope to finish writing sometime. But also if anyone else is reading and loves this kind of stuff (especially if you've solved the challenge), I am taking co-authors!

### First, the problem statement:

Simultaneously solve ten equations of the form
$$(k + i)(x_i + A_i) \equiv 1 \pmod{p},$$
where $k \in \mathbb{F}$ is unknown, $x_i \in \{0, 1, \ldots, 2^{170}-1\}$ are small and unknown small, $A_i$ are known, and $i = 1,2,\ldots,10$.

Specifically, for `zerodaycrypto` we have the values $p = 2^{255}-19$ and 

$$\mathbf{A} = (29431621415867921698671444, 12257315102018176664717361, 6905311467813097279935853, 13222913226749606936127836, 25445478808277291772285314, 9467767007308649046406595, 33796240042739223741879633, 520979008712937962579001, 31472015353079479796110447, 38623718328689304853037278, 17149222936996610613276307, 21988007084256753502814588, 11696872772917077079195865, 6767350497471479755850094) \times 2^{170}.$$

(The flag is then the value of $k+1$ because of 1-indexing. Maybe I should have 0-indexed instead.)

In other words, we have an overdetermined system with 11 unknowns and 10 variables, so knowing any one value determines the others. But we want to solve them simultaneously such that all the $x_i$ are small, which is where Coppersmith comes in.

### Constructing the Coppersmith polynomials

Let's start with some small and boring polynomials first to see what _not_ to do. If we take the resultant of
$$(k + i)(x_i + A_i)-1, i \in \{1,2\}$$
to eliminate $k$, we get the polynomial equation
$$x_1 x_2 + (A_2-1)x_1 + (A_1+1)x_2 + (A_1 A_2 - A_1 + A_2) \equiv 0 \pmod{p}.$$

This is a polynomial equation of degree 2 giving rise to a root mod $p$.

We can also square this for example, or multiply it with the similar $x_3 x_4$ equation, to get a polynomial equation of degree 4 with a root mod $p^2$. But turns out these polynomials are weak and we don't want them.

We say that a polynomial of degree $d$ is **good** if produces a root mod $p^{d-1}$. So the degree 2 polynomial is good, but the degree 4 is not.

How do we construct good polynomials? Using determinants! Consider this 6x6 matrix
$$
M =
\begin{pmatrix}
  (x_1+A_1) & 1(x_1+A_1)-1   &   1(x_1+A_1) &  1(x_1+A_1)-1   &    1(x_1+A_1) &   1(x_1+A_1)- 1 \\
  (x_2+A_2) & 2(x_2+A_2)-1   &   2(x_2+A_2) &  4(x_2+A_2)-2   &    4(x_2+A_2) &   8(x_2+A_2)- 4 \\
  (x_3+A_3) & 3(x_3+A_3)-1   &   3(x_3+A_3) &  9(x_3+A_3)-3   &    9(x_3+A_3) &  27(x_3+A_3)- 9 \\
  (x_4+A_4) & 4(x_4+A_4)-1   &   4(x_4+A_4) & 16(x_4+A_4)-4   &   16(x_4+A_4) &  64(x_4+A_4)-16 \\
  (x_5+A_5) & 5(x_5+A_5)-1   &   5(x_5+A_5) & 25(x_5+A_5)-5   &   25(x_5+A_5) & 125(x_5+A_5)-25 \\
  (x_6+A_6) & 6(x_6+A_6)-1   &   6(x_6+A_6) & 36(x_6+A_6)-6   &   36(x_6+A_6) & 216(x_6+A_6)-36 \\
\end{pmatrix}
$$

We can add $k$ times the first column to the second column. This is a pivot operation that doesn't change the determinant. But also the effect of this operation is that the second column are now all multiples of $p$. Likewise with the third and fourth columns. Then again with the fifth and six columns.

This means we can say that $det(M)$ is a polynomial with a root mod $p^3$.

Which is not useful in itself if $det(M)$ has degree 6. But what if we pivot different pairs of columns instead? In particular, if we subtract the third column from the second, and the fourth column from the third, we will get

$$
M' =
\begin{pmatrix}
  (x_1+A_1) & -1   &   1(x_1+A_1) & -1   &    1(x_1+A_1) &   1(x_1+A_1)- 1 \\
  (x_2+A_2) & -1   &   2(x_2+A_2) & -2   &    4(x_2+A_2) &   8(x_2+A_2)- 4 \\
  (x_3+A_3) & -1   &   3(x_3+A_3) & -3   &    9(x_3+A_3) &  27(x_3+A_3)- 9 \\
  (x_4+A_4) & -1   &   4(x_4+A_4) & -4   &   16(x_4+A_4) &  64(x_4+A_4)-16 \\
  (x_5+A_5) & -1   &   5(x_5+A_5) & -5   &   25(x_5+A_5) & 125(x_5+A_5)-25 \\
  (x_6+A_6) & -1   &   6(x_6+A_6) & -6   &   36(x_6+A_6) & 216(x_6+A_6)-36 \\
\end{pmatrix},
$$

and now the same polynomial has degree 4. In fact, if we translate it by $-\mathbf{A}$, we now have

$$
M'' =
\begin{pmatrix}
  x_1 & -1   &   1x_1 & -1   &    1x_1 &   1x_1- 1 \\
  x_2 & -1   &   2x_2 & -2   &    4x_2 &   8x_2- 4 \\
  x_3 & -1   &   3x_3 & -3   &    9x_3 &  27x_3- 9 \\
  x_4 & -1   &   4x_4 & -4   &   16x_4 &  64x_4-16 \\
  x_5 & -1   &   5x_5 & -5   &   25x_5 & 125x_5-25 \\
  x_6 & -1   &   6x_6 & -6   &   36x_6 & 216x_6-36 \\
\end{pmatrix}
$$

This is now a different polynomial, but it's clear (from swapping columns, and multiplying by -1 if necessary) that $det(M'')$ is the sum of a $(6,4)$-beetle and a $(6,3)$-beetle. This means that $det(M'') \in \mathrm{SBG}$, and thus $det(M) \in \mathrm{SBG}$.

This gives us a quick-and-easy way to construct lots of good polynomials! Actually many of them are linearly dependent, so what we can do is fix a superbeetle basis for $\mathrm{SBG}_{10}$, and then for each element of the basis we construct a corresponding polynomial by taking the same values of $i$ as its beetleshell.

To be extra specific, we construct the following:
- We have exactly one degree-6 polynomial mod $p^5$. This uses $\mathrm{i} = (1,2,3,4,5,6,7,8,9,10)$.
- We have 21 degree-5 polynomials mod $p^4$. The uses $\mathrm{i} = (1,2,3,4,5,8,9,10), (1,2,3,4,6,8,9,10), \ldots$ with the last three elements fixed as $(8,9,10)$.
- We have 70 degree-4 polynomials mod $p^3$. The uses $\mathrm{i} = (1,2,3,4,9,10), \ldots$ with the last two elements fixed as $(9,10)$.
- We have 84 degree-3 polynomials mod $p^2$. The uses $\mathrm{i} = (1,2,3,10), \ldots$ with the last one elements fixed as $(10)$.
- We have 45 degree-2 polynomials mod $p^1$. The uses $\mathrm{i} = (1,2), \ldots$ with no fixed elements.
- We have 10 degree-1 polynomials mod $1$.
- We have 1 degree-0 polynomial mod $1$.

Actually, those last eleven probably don't even really count as equations. But anyway this gives us our lattice of dimension 232, and we are done!

We can do our usual Coppersmith analysis on this easily since it's a triangular matrix, and with this matrix you only need the high $\frac{231}{743} \simeq 0.311$ bits (so a lot better than one third), but I leave the details to the paper.

## Sage implementation

We will start by first ignoring the $\mathbf{A}$ completely. This is because SBG is invariant under translation, so will just translate it at the end.

In [1]:
def flatter(M):
    from subprocess import check_output
    from re import findall
    z = "[[" + "]\n[".join(" ".join(map(str, row)) for row in M) + "]]"
    ret = check_output(["flatter", "-rhf", "1.03"], input=z.encode())
    return matrix(M.nrows(), M.ncols(), map(ZZ, findall(b"-?\\d+", ret)))
    
N, Δ, p = 10, 2^170, 2^255 - 19
F = PolynomialRing(QQ, 'x', N, order='deglex')
PP = p ^ (N//2)

class Monomial:
    def __init__(self, Ω):
        K = len(Ω)//2
        lhs, rhs = F(1), F(0)
        if Ω:
            f = lambda k: sum(prod(F.gen(i) / prod(i-j for j in Ω-S) for i in S) for S in Subsets(Ω, k))
            lhs, rhs = f(K+1), (-1)^K * f(K)
            
        self.top = lhs.numerator()
        self.eqn = (lhs - rhs) / lhs.lc()
        self.mul = p^K
        
def cm(arr):
    return Sequence(arr).coefficients_monomials(sparse=False)
    
def dm(arr):
    return diagonal_matrix(arr, sparse=False)

H = N//2 + 1
subsets = [x-{N} + Set(range(H+i, N)) for i in range(H) for x in Subsets(range(H+i), H-i)]
tups = [Monomial(s) for s in sorted(subsets)[::-1]]
print(f'{subsets = }')

subsets = [{0, 1, 2, 3, 4, 5, 6, 7, 8, 9}, {0, 1, 2, 3, 4, 7, 8, 9}, {0, 1, 2, 3, 5, 7, 8, 9}, {0, 1, 2, 3, 6, 7, 8, 9}, {0, 1, 2, 4, 5, 7, 8, 9}, {0, 1, 2, 4, 6, 7, 8, 9}, {0, 1, 2, 5, 6, 7, 8, 9}, {0, 1, 3, 4, 5, 7, 8, 9}, {0, 1, 3, 4, 6, 7, 8, 9}, {0, 1, 3, 5, 6, 7, 8, 9}, {0, 1, 4, 5, 6, 7, 8, 9}, {0, 2, 3, 4, 5, 7, 8, 9}, {0, 2, 3, 4, 6, 7, 8, 9}, {0, 2, 3, 5, 6, 7, 8, 9}, {0, 2, 4, 5, 6, 7, 8, 9}, {0, 3, 4, 5, 6, 7, 8, 9}, {1, 2, 3, 4, 5, 7, 8, 9}, {1, 2, 3, 4, 6, 7, 8, 9}, {1, 2, 3, 5, 6, 7, 8, 9}, {1, 2, 4, 5, 6, 7, 8, 9}, {1, 3, 4, 5, 6, 7, 8, 9}, {2, 3, 4, 5, 6, 7, 8, 9}, {0, 1, 2, 3, 8, 9}, {0, 1, 2, 4, 8, 9}, {0, 1, 2, 5, 8, 9}, {0, 1, 2, 6, 8, 9}, {0, 1, 2, 7, 8, 9}, {0, 1, 3, 4, 8, 9}, {0, 1, 3, 5, 8, 9}, {0, 1, 3, 6, 8, 9}, {0, 1, 3, 7, 8, 9}, {0, 1, 4, 5, 8, 9}, {0, 1, 4, 6, 8, 9}, {0, 1, 4, 7, 8, 9}, {0, 1, 5, 6, 8, 9}, {0, 1, 5, 7, 8, 9}, {0, 1, 6, 7, 8, 9}, {0, 2, 3, 4, 8, 9}, {0, 2, 3, 5, 8, 9}, {0, 2, 3, 6, 8, 9}, {0, 2, 3, 7, 8, 9}, {0, 2, 4, 5, 8, 9}, {0, 2, 4, 6

What happened here is I've created 232 subsets of {0,1,...,9} as described. This is used for both the 232 basis polynomials of $\mathrm{SBG}_{10}$, as well as the 232 (pre-translated) Coppersmith polynomials for the lattice.

Instead of directly using the determinants of matrix, I've cheated a bit by using products of differences. This is basically because it's a [Vandermonde-like matrix](https://en.wikipedia.org/wiki/Vandermonde_matrix). Same thing, just faster.

Now let's translate by $\mathbf{A}$ and turn it into a matrix.

In [2]:
A = [29431621415867921698671444, 12257315102018176664717361, 6905311467813097279935853, 13222913226749606936127836, 25445478808277291772285314, 9467767007308649046406595, 33796240042739223741879633, 520979008712937962579001, 31472015353079479796110447, 38623718328689304853037278, 17149222936996610613276307, 21988007084256753502814588, 11696872772917077079195865, 6767350497471479755850094]
M = cm(vector([t.eqn for t in tups])([g*Δ+x for g,x in zip(A, F.gens())]))[0]
print(f'{M.dimensions() = }')

M.dimensions() = (232, 848)


Ok, I've cheated again by having a non-square matrix. This is because `cm` decomposes the polynomials wrt to full monomial basis. We're supposed to find the coordinates using our SBG basis, but also this turns out be the same as just removing columns which is so much easier.

In [3]:
M = M[:, M.pivots()]
print(f'{M.dimensions() = }')

M.dimensions() = (232, 232)


Remember how the polynomials are all taken modulo different powers of $p$? To make the lattice consistent, we multiply each row by some power of $p$ (inversely proportional to its strength), so that _the entire system_ is now modulo the same power of $p$.

In [4]:
WD = dm([t.top.lc() for t in tups])
M2 = matrix([row % t.mul / t.mul * PP for t, row in zip(tups, WD * M * WD^-1)])
M2[-N-1:,-N-1:] = identity_matrix(N + 1) * PP
print(f'{M2.dimensions() = }')

M2.dimensions() = (232, 232)


We now balance our lattice with the weights of the basis polynomials. In a standard Coppersmith lattice, a monomial like $x_1 x_3$ would have weight $\Delta^2$. Now our basis polynomials are no longer monomials but homogeneous polynomials so they look slightly more complicated, but still not terrible, e.g. $x_1 x_3 + x_2 x_4$ would just give $2\Delta^2$.

In [5]:
T = walltime()
WT = dm([sum(abs(x) for x,_ in t.top) * Δ^t.top.degree() for t in tups])
M3 = flatter(M2._clear_denom()[0] * WT) * WT^-1
print(f'{walltime(T) = }')

walltime(T) = 45.30013179779053


And it is flatter-reduced! All that's left is to solve the system in the reals.

Because all our polynomials were good (except the last one), heuristically we can use all of them (except the last one). This means we can skip the Groebner basis and just `solve_right` directly.

In [6]:
v = M3.solve_right(vector([0]*(M3.nrows()-1)+[1]))[-N-1:]
k = 1/mod(A[0] * Δ + v[0]/v[-1], p)
print(f'{k = }')

k = 50850654797340916877839991041176893622674697204475566066988576396536406370424


And we have recovered `k`! Let's just check that we get $\mathbf{A}$ back.

In [7]:
assert [1/(k+i) >> 170 for i in range(1+3+3+7)] == A

At this point we've been more mathematician than crypto player, so the flag doesn't matter and we're content to have solved for $k$. QED.

But just for completeness anyway:

In [8]:
print(f'SEKAI{{{k.to_bytes().decode()}}}')

SEKAI{pls_help_me_write_the_paper_kthx}
